In [1]:
!pip install transformers==4.33 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.0 which is incompatible.


In [2]:
import torch
import gc
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoConfig

In [3]:
LANGUAGE_TARGET_LABEL = 'nivkh_Cyrl'

def fix_tokenizer(tokenizer, new_lang=LANGUAGE_TARGET_LABEL):
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset
    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [4]:
model_name = "el-izm/nllb_nivkh_rus_600M_base"
# model_name = "el-izm/nllb_nivkh_rus_1.3B_base"
# model_name = "el-izm/nllb_nivkh_rus_600M_extended"
# model_name = "el-izm/nllb_nivkh_rus_600M_amur_extended"
# model_name = "el-izm/nllb_nivkh_rus_600M_sakh_extended"

cfg = AutoConfig.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=cfg).to('cuda')
tokenizer = NllbTokenizer.from_pretrained(model_name)
fix_tokenizer(tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.01M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def translate(text, src_lang, tgt_lang, max_length='auto', num_beams=4):
    lang_map = {
        'N': 'nivkh_Cyrl',
        'R': 'rus_Cyrl'
    }
    src_lang_code = lang_map[src_lang]
    tgt_lang_code = lang_map[tgt_lang]
    tokenizer.src_lang = src_lang_code
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    if max_length == 'auto':
        max_length = int(32 + 2.0 * encoded.input_ids.shape[1])
    model.eval()
    with torch.no_grad():
        generated_tokens = model.generate(
            **encoded.to(model.device),
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang_code],
            max_length=max_length,
            num_beams=num_beams,
            no_repeat_ngram_size=4
        )
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return result

In [8]:
text = 'Ӿы нивх лу?'
src = 'N'
trg = 'R'
result = translate(text, src, trg)
print(result)

Этого человека?


In [9]:
def cleanup():
    # для чистки CUDA при смене модели
    gc.collect()
    torch.cuda.empty_cache()
cleanup()